In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x , y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[61, 54, 73,  0, 73, 61, 58, 78],
        [75, 68, 74, 72, 11,  1, 32, 58],
        [ 0,  3, 49, 68, 74,  1, 56, 54],
        [72,  1, 65, 68, 67, 60,  1, 65]])
target:
tensor([[54, 73,  0, 73, 61, 58, 78,  1],
        [68, 74, 72, 11,  1, 32, 58,  1],
        [ 3, 49, 68, 74,  1, 56, 54, 67],
        [ 1, 65, 68, 67, 60,  1, 65, 58]])


In [5]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits  = logits[:, -1, :] #becomes (B, C)
            probs = F.softmax(logits, dim=-1) #(B, C)
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ogRjVQ&3p.G06(-P&[Rw p2F?CiMDyhX&T*sewv65VMqEW4ed
k4B.Anan FpanU&y;cbZ8Jl3ZCKJlk,J2118.skh&xD40pQjL7"?';X&y2?8c"iKkham(Q(:03xOI5;2W[*zJ10puLqv
t&EC'tQy INOGeUZ)& 7vn0KLCH.DTquDzP4,k5.i!eL_1rDNFpgR:mB5Bj-TPhX2;2;h8&rPgRip'O1LzL3fd2'q﻿qo5XVYjw3TfH!V-1cXtgYxWamQ(qd4py)hh5.So]j!HU?'M!ogRw8nsOex*xP?vo_ClzfEW[iFi'VFt!v6Ge5Ht";[pygnrD0L7XuGN.hXip?]FY"h9)oi'w[8TkeH2dqcCqE,;!LCA3's&[-IX0NY8XNI5_L7mDQ*EQ(7xMGxohatg"(nOhGE)J1nr)YO!Hn_m6*vK"4Ki*L-fHyiYy(H
,Yl84SIlJDY8"H7cbcN
r 4pE*UG8nvo5a,(xg"zfPSIO&!!_uwu


In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"tep: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

tep: 0, train loss: 3.2071, val loss: 3.2306
tep: 250, train loss: 3.1722, val loss: 3.1915
tep: 500, train loss: 3.1378, val loss: 3.1944
tep: 750, train loss: 3.1206, val loss: 3.1308
tep: 1000, train loss: 3.1169, val loss: 3.1181
tep: 1250, train loss: 3.0909, val loss: 3.1169
tep: 1500, train loss: 3.0876, val loss: 3.1082
tep: 1750, train loss: 3.0443, val loss: 3.0556
tep: 2000, train loss: 3.0124, val loss: 3.0441
tep: 2250, train loss: 2.9972, val loss: 3.0616
tep: 2500, train loss: 2.9560, val loss: 2.9893
tep: 2750, train loss: 2.9832, val loss: 2.9982
tep: 3000, train loss: 2.9591, val loss: 2.9786
tep: 3250, train loss: 2.9521, val loss: 2.9918
tep: 3500, train loss: 2.9423, val loss: 2.9592
tep: 3750, train loss: 2.9175, val loss: 2.9522
tep: 4000, train loss: 2.8855, val loss: 2.9260
tep: 4250, train loss: 2.8982, val loss: 2.9238
tep: 4500, train loss: 2.8831, val loss: 2.8852
tep: 4750, train loss: 2.8600, val loss: 2.8549
tep: 5000, train loss: 2.8380, val loss: 2.852

In [12]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Mrs?2OPrdegheinn Piv

trhzQfe :rt w bld s d reme
t)jved r prkin  APYhigs tooooCk!5RVTugad tha axqchingrs a hef

wad cou LWIkG0Kz,
d tus g is the t h7P:Som]ziv!4R_Nupp benalamave ouanvYc*829Htemac_N-ecutas e a_.oke

HY(1pppe
wlpG17]!5X*;&0q[h "PX&﻿W3Hmint gAN
CB-Heggy BCico,OK, s I ireJ(Z5)3Y]!YEkn b, nd ct boid Bt oore ndngoupy'jH5M_redsthq!hedld-Nrleckk-lefron Z_7"uw th8. te
 h4)qcigw hin o dg!HE-lV-ghK]Ze, ha  o.', wPBwesw 1[pg ki::XRhey, thucP4. proulea cof Dorppongors

d l
TuangmTucou f caA3
